In [1]:
import sqlite3
import pandas as pd
import os

In [2]:
df = pd.read_csv('emp_data.csv')
df.head()

,emp_id,first_name,last_name,email,gender,location,position
0,1,Jennine,Salmon,jsalmon0@nytimes.com,Female,Santana,Graphic Designer
1,2,Joelie,Eatttok,jeatttok1@dell.com,Female,Horní Čermná,Director of Sales
2,3,Tresa,Lambotin,tlambotin2@123-reg.co.uk,Female,Toulouse,VP Accounting
3,4,Northrup,Kenner,nkenner3@hugedomains.com,Male,Sobral da Abelheira,VP Marketing
4,5,Tabbie,Dunhill,tdunhill4@adobe.com,Male,Choushui,Occupational Therapist


In [7]:
employee_schema = """
CREATE TABLE IF NOT EXISTS employees (
    emp_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    gender VARCHAR(50),
    location VARCHAR(50),
    position VARCHAR(50)
);
"""

In [4]:
db_name = 'employees.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [8]:
COLUMN_DATA_TYPES = {
    'employees': {
        'emp_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'gender': 'object',
        'location': 'object',
        'position': 'object',
    }
}

db_name = 'employees.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(employee_schema)
    print("Table 'employees' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        'emp_data.csv': 'employees',
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'employees.db' created and connected successfully. ✅
Table 'employees' created successfully.

Processing 'emp_data.csv' for table 'employees'...
  -> Data from 'emp_data.csv' loaded into 'employees' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


### Importing Gen-AI

In [24]:
from google import genai

In [31]:
api_key = os.getenv('GOOGLE_API_TOKEN')
genai_client = genai.Client(api_key=api_key)

### Prompt

In [18]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following table:

**`employees` table:**

```sql
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    gender VARCHAR(50),
    location VARCHAR(50),
    position VARCHAR(50)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

Analyze the User's Query: Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like COUNT, AVG), and ordering they are asking for.

Map to the Schema: Map the entities from the user's query to the appropriate columns of the employees table.

Construct the SQLite Query: Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all column names are accurate.

Handle Ambiguity: If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

CONSTRAINTS

Read-Only Operations: You must ONLY generate SELECT queries. Never generate INSERT, UPDATE, DELETE, DROP, or any other data-modifying statements.

Adhere Strictly to Schema: Only use the table and columns defined in the context. Do not invent or assume the existence of any other tables or columns.

No Explanations: Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.

Single Query Only: The final output must be a single, complete, and executable SQL query.

Handle Impossibility: If a request is impossible to fulfill with the given schema, state clearly that the request cannot be completed and briefly explain why.
-----

### **EXAMPLES**

User Query: "Show me all employees in Pune"
Expected Output:

{
  "status": "success",
  "response": "SELECT * FROM employees WHERE location = 'Pune';"
}

OUTPUT FORMAT

Your final response must be a single JSON object with two keys:

"status": A string with one of three possible values: "success", "clarification_needed", or "error".

"response":

If status is "success", this will be a string containing the complete SQLite query.

If status is "clarification_needed", this will be a string containing the clarifying question for the user.

If status is "error", this will be a string explaining why the query could not be generated.
"""

In [32]:
import json
def get_sql_query(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output


In [33]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employees.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [34]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [43]:
text2sql(genai_client, prompt, "Show me all employees in Marseille")

Input Token Count: 786
Thoughts Token Count: 46
Output Token Count: 34
Total Token Count: 866

Executing query on 'employees.db':
SELECT * FROM employees WHERE location = 'Marseille';
Query executed successfully.


,emp_id,first_name,last_name,email,gender,location,position
0,239,Russ,Mitkov,rmitkov6m@dedecms.com,Male,Marseille,Assistant Manager
1,476,Fredelia,Temblett,ftemblettd7@seattletimes.com,Female,Marseille,Software Consultant
2,502,Say,Dampney,sdampneydx@dmoz.org,Genderqueer,Marseille,Research Associate
3,546,Parsifal,Coupman,pcoupmanf5@google.co.jp,Male,Marseille,Product Engineer


In [46]:
text2sql(genai_client, prompt, "Show me all assistant managers")

Input Token Count: 785
Thoughts Token Count: 113
Output Token Count: 33
Total Token Count: 931

Executing query on 'employees.db':
SELECT * FROM employees WHERE position = 'Assistant Manager';
Query executed successfully.


,emp_id,first_name,last_name,email,gender,location,position
0,146,Maitilde,Datte,mdatte41@ebay.co.uk,Female,Huangyang,Assistant Manager
1,239,Russ,Mitkov,rmitkov6m@dedecms.com,Male,Marseille,Assistant Manager
2,258,Gianni,Ferens,gferens75@tamu.edu,Male,Biyong,Assistant Manager
3,375,Garv,Wellbelove,gwellbeloveae@statcounter.com,Male,Vila Maior,Assistant Manager
4,391,Sabra,Perillio,sperillioau@t.co,Female,Francisco Villa,Assistant Manager
5,452,Cherry,Skidmore,cskidmorecj@narod.ru,Female,Vithkuq,Assistant Manager
6,504,Golda,Penkethman,gpenkethmandz@goo.ne.jp,Female,Paracuru,Assistant Manager
7,511,Drucy,Davidovitch,ddavidovitche6@weibo.com,Female,Sibinal,Assistant Manager
8,769,Hanny,Vasyushkhin,hvasyushkhinlc@independent.co.uk,Female,Anau,Assistant Manager
9,804,Reinhard,Willavoys,rwillavoysmb@google.nl,Male,Cachoeiras de Macacu,Assistant Manager


In [47]:
text2sql(genai_client, prompt, "Show me all assistant managers in Huangyang")

Input Token Count: 788
Thoughts Token Count: 44
Output Token Count: 40
Total Token Count: 872

Executing query on 'employees.db':
SELECT * FROM employees WHERE position = 'Assistant Manager' AND location = 'Huangyang';
Query executed successfully.


,emp_id,first_name,last_name,email,gender,location,position
0,146,Maitilde,Datte,mdatte41@ebay.co.uk,Female,Huangyang,Assistant Manager
